In [1]:
import requests
import datetime
data = requests.get('https://website-api.ardshinbank.am/currency').json()
data

{'data': {'currencies': {'cash': [{'type': 'USD',
     'buy': '390.5',
     'sell': '396',
     'cb': '393.69'},
    {'type': 'EUR', 'buy': '405', 'sell': '419', 'cb': '411.09'},
    {'type': 'RUR', 'buy': '4.35', 'sell': '4.75', 'cb': '4.4047'},
    {'type': 'GBP', 'buy': '491', 'sell': '513', 'cb': '497.94'},
    {'type': 'CHF', 'buy': '429', 'sell': '452', 'cb': '436.66'}],
   'no_cash': [{'type': 'USD', 'buy': '390.5', 'sell': '396', 'cb': '393.69'},
    {'type': 'EUR', 'buy': '405', 'sell': '419', 'cb': '411.09'},
    {'type': 'RUR', 'buy': '4.23', 'sell': '4.75', 'cb': '4.4047'},
    {'type': 'GBP', 'buy': '491', 'sell': '513', 'cb': '497.94'},
    {'type': 'CHF', 'buy': '429', 'sell': '452', 'cb': '436.66'},
    {'type': 'AED', 'buy': '104', 'sell': '109', 'cb': '107.2'},
    {'type': 'AUD', 'buy': '238', 'sell': '251', 'cb': '244.99'},
    {'type': 'CAD', 'buy': '265', 'sell': '279', 'cb': '272.88'},
    {'type': 'CNY', 'buy': '52.7', 'sell': '55.2', 'cb': '53.998'},
    {'type

In [10]:
data['updatedAt']

'2025-03-04T05:56:42.266Z'

In [12]:
data2 = requests.get('https://api.tinkoff.ru/v1/currency_rates?from=USD&to=RUB').json()
data2

{'trackingId': 'GWVWGH9YWU',
 'resultCode': 'OK',
 'payload': {'lastUpdate': {'milliseconds': 1741068584401},
  'rates': [{'category': 'DepositPayments',
    'fromCurrency': {'code': 840, 'name': 'USD', 'strCode': '840'},
    'toCurrency': {'code': 643, 'name': 'RUB', 'strCode': '643'},
    'buy': 83.75,
    'sell': 94.5},
   {'category': 'CreditCardsOperations',
    'fromCurrency': {'code': 840, 'name': 'USD', 'strCode': '840'},
    'toCurrency': {'code': 643, 'name': 'RUB', 'strCode': '643'},
    'buy': 83.75,
    'sell': 94.5},
   {'category': 'DebitCardsOperations',
    'fromCurrency': {'code': 840, 'name': 'USD', 'strCode': '840'},
    'toCurrency': {'code': 643, 'name': 'RUB', 'strCode': '643'},
    'buy': 83.75,
    'sell': 94.5},
   {'category': 'PrepaidCardsOperations',
    'fromCurrency': {'code': 840, 'name': 'USD', 'strCode': '840'},
    'toCurrency': {'code': 643, 'name': 'RUB', 'strCode': '643'},
    'buy': 83.75,
    'sell': 94.5},
   {'category': 'C2CTransfers',
    'fr

In [18]:
next((x for x in data2['payload']['rates'] if x['category'] == 'DebitCardsTransfers'))['sell']

98.15

In [24]:
from datetime import datetime, timezone, timedelta
import requests
import logging

class BaseAPIClient:
    def __init__(self):
        self.name = type(self).__name__
        self.logger = logging.getLogger(self.name)
        self.msk_offset = timedelta(hours=3)  # MSK = UTC+3

    def parse_timestamp(self, ts):
        """Конвертирует разные форматы времени в (ISO строка MSK, Unix timestamp MSK)"""
        try:
            dt_utc = None

            # Обработка числовых timestamp (секунды/миллисекунды)
            if isinstance(ts, (int, float)):
                if ts > 1e12:  # Миллисекунды
                    ts_seconds = ts / 1000
                else:  # Секунды
                    ts_seconds = ts
                dt_utc = datetime.fromtimestamp(ts_seconds, tz=timezone.utc)

            # Обработка строковых форматов
            elif isinstance(ts, str):
                ts_clean = ts.rstrip('Z').replace('Z', '')
                dt = datetime.fromisoformat(ts_clean)

                if dt.tzinfo:  # Конвертируем в UTC если есть смещение
                    dt_utc = dt.astimezone(timezone.utc)
                else:  # Предполагаем UTC если нет информации
                    dt_utc = dt.replace(tzinfo=timezone.utc)

            else:
                raise ValueError(f"Unsupported timestamp type: {type(ts)}")

            # Конвертация в московское время (UTC+3)
            dt_msk = dt_utc + self.msk_offset

            # Форматируем результат
            iso_msk = dt_msk.isoformat(timespec='milliseconds') + '+03:00'
            unix_msk = int(dt_utc.timestamp())

            return iso_msk, unix_msk

        except Exception as e:
            self.logger.error(f"Timestamp parse error: {e}")
            now = datetime.now(timezone.utc) + self.msk_offset
            return now.isoformat(timespec='milliseconds') + '+03:00', int(now.timestamp())

In [25]:
cls = BaseAPIClient()

In [29]:
cls.parse_timestamp(1741071831222)

('2025-03-04T10:03:51.222+00:00+03:00', 1741071831)